# Model and data import

Options for models:

1) Glycolysis + Pyk mutation
2) Trp

# Data Analysis with kinetic and thermodynamic models

## Comparing estimated and real enzyme concentrations

## Analyzing metabolomics data

### Calculating thermodynamic distance from equilibrium

### Calculating enzyme bound states

# Simulating kinetic models for strain design

## Enzyme concentration perturbations